In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# Hansard parser from XML files saved to disk


In [2]:
def read_xml_from_path(path):
    # Read from path - decode bytes to strings
    with open(path, 'rb') as file:
        # Replace unrecognised utf-8 tokens with '?'
        xml = file.read().decode(errors='replace')

    return xml

In [3]:
def parse_hansard_xml(xml, memberid2personid_lookup, filter_no_speaker=True):
    # Use BeautifulSoup to tidy the XML
    import pandas as pd
    from bs4 import BeautifulSoup
    bs = BeautifulSoup(xml)

    # Define a function to parse a chunk of XML for an individual speech
    def parse_speech_xml(speech_xml):
        name = speech_xml.get('speakername')
        speech_id = speech_xml.get('id')
        person_id = speech_xml.get('person_id')
        if pd.isna(person_id):
            member_id = speech_xml.get('speakerid')
            if member_id in memberid2personid_lookup.keys():
                person_id = memberid2personid_lookup[member_id]
        text = speech_xml.find('p')
        if text is not None:
            text = text.get_text()
        return name, speech_id, person_id, text

    # Get an XML object for each speech in the debate
    speeches = bs.find_all('speech')

    # Create dictionary to store debate features
    debate_dict = {'name': [],
                   'speech_id': [],
                   'person_id': [],
                   'text': []}

    for speech_xml in speeches:
        name, speech_id, person_id, text = parse_speech_xml(speech_xml)
        if filter_no_speaker:  # If we're filtering 'no speaker' lines, then drop speeches with no name
            if name is not None:
                for variable in debate_dict.keys():
                    debate_dict[variable].append(eval(variable))
        else:
            for variable in debate_dict.keys():
                debate_dict[variable].append(eval(variable))

    return pd.DataFrame(debate_dict)

In [5]:
people_df = pd.read_csv('people.csv').drop('Unnamed: 0', axis=1)


def lookup(row): return {
    member_id: row.person_id for member_id in eval(row.memberships)}


lookup_list = people_df.progress_apply(lookup, axis=1).to_list()

memberid2personid_lookup = {
    member_id: person_id for lookup_dict in lookup_list for member_id, person_id in lookup_dict.items()}

target_urls = pd.read_csv('debate_urls.csv').url.to_list()

df = parse_hansard_xml('', memberid2personid_lookup)

for url in tqdm(target_urls):
    filename = 'debates_xml/'+url.split('/')[-1]

    speech_xml = read_xml_from_path(filename)
    debate_df = parse_hansard_xml(speech_xml, memberid2personid_lookup)

    df = pd.concat([df, debate_df], ignore_index=True)

display(df.head())

 11%|█▏        | 2137/18877 [03:48<24:35, 11.34it/s]  /Users/felixwallis/Desktop/UCL Politics and International Relations Course/Dissertation/pir-dissertation/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 18877/18877 [2:55:58<00:00,  1.79it/s]   


,name,speech_id,person_id,text
0,Colonel F. B. MILDMAY (addressing himself to t...,uk.org.publicwhip/debate/1919-02-04a.3.1,uk.org.publicwhip/person/15282,"Sir Courtenay Ilbert, in accordance with the G..."
1,Sir HENRY DALZIEL,uk.org.publicwhip/debate/1919-02-04a.5.0,unknown,"Sir Courtenay Ilbert, I am fully conscious tha..."
2,Mr. LOWTHER (who was received with general che...,uk.org.publicwhip/debate/1919-02-04a.8.0,unknown,"Sir Courtenay Ilbert,—In accordance with the a..."
3,Mr. SPEAKER-ELECT (standing on the upper step ...,uk.org.publicwhip/debate/1919-02-04a.11.0,unknown,"Before taking the Chair, once more as Speaker ..."
4,Mr. BONAR LAW (Leader of the House),uk.org.publicwhip/debate/1919-02-04a.11.3,uk.org.publicwhip/person/20693,"Mr. Speaker-Elect,—In the unavoidable absence ..."


In [6]:
def date_from_speech_id(x): return pd.to_datetime(x.split('/')[-1][:10])


df['speech_date'] = df.speech_id.progress_apply(
    lambda x: date_from_speech_id(x))

100%|██████████| 5848603/5848603 [08:49<00:00, 11048.62it/s]


In [7]:
display(df)

,name,speech_id,person_id,text,speech_date
0,Colonel F. B. MILDMAY (addressing himself to t...,uk.org.publicwhip/debate/1919-02-04a.3.1,uk.org.publicwhip/person/15282,"Sir Courtenay Ilbert, in accordance with the G...",1919-02-04
1,Sir HENRY DALZIEL,uk.org.publicwhip/debate/1919-02-04a.5.0,unknown,"Sir Courtenay Ilbert, I am fully conscious tha...",1919-02-04
2,Mr. LOWTHER (who was received with general che...,uk.org.publicwhip/debate/1919-02-04a.8.0,unknown,"Sir Courtenay Ilbert,—In accordance with the a...",1919-02-04
3,Mr. SPEAKER-ELECT (standing on the upper step ...,uk.org.publicwhip/debate/1919-02-04a.11.0,unknown,"Before taking the Chair, once more as Speaker ...",1919-02-04
4,Mr. BONAR LAW (Leader of the House),uk.org.publicwhip/debate/1919-02-04a.11.3,uk.org.publicwhip/person/20693,"Mr. Speaker-Elect,—In the unavoidable absence ...",1919-02-04
...,...,...,...,...,...
5848598,Nickie Aiken,uk.org.publicwhip/debate/2024-01-11a.551.0,uk.org.publicwhip/person/25817,It was a pleasure to allow that intervention.,2024-01-11
5848599,Nigel Evans,uk.org.publicwhip/debate/2024-01-11a.551.1,uk.org.publicwhip/person/10190,"I have been an MP for 31 years, and it was not...",2024-01-11
5848600,Elliot Colburn,uk.org.publicwhip/debate/2024-01-11a.552.2,uk.org.publicwhip/person/25907,"I thank the Speaker’s Office and you, Mr Deput...",2024-01-11
5848601,Bim Afolami,uk.org.publicwhip/debate/2024-01-11a.554.0,uk.org.publicwhip/person/25661,I congratulate my hon. Friend the Member for C...,2024-01-11


In [8]:
df.to_csv('hansard_in_full.csv')